In [1]:
!pip install tslearn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 16.6 MB/s eta 0:00:00


# Importar librerias

In [2]:
# Conectar al entorno de Collab las carpetas de Google Drive
ColabNotebook = 'google.colab' in str(get_ipython())

if ColabNotebook:
    # monta G-drive en entorno COLAB
    from google.colab import drive
    drive.mount('/content/drive/')

    # carpeta donde se encuentran archivos .py auxiliares
    FUENTES_DIR = '/content/drive/MyDrive/Colab Notebooks/FUENTES/'
    DATOS_DIR = '/content/drive/MyDrive/Colab Notebooks/DATOS/'      # carpeta donde se encuentran los datasets
    LOCAL_DIR = './DATOS/'  # carpeta local en maquina virtual
    SCALERS_DIR = '/content/drive/MyDrive/Colab Notebooks/SCALERS/'
    ENCODERS_DIR = '/content/drive/MyDrive/Colab Notebooks/ENCODERS/'
    OUTPUTS_DIR = '/content/drive/MyDrive/Colab Notebooks/OUTPUTS/'
else:
    # configuración para notebook con instalación LOCAL
    FUENTES_DIR = '../../FUENTES'         # carpeta donde se encuentran archivos .py auxiliares
    DATOS_DIR   = '../../DATOS/' # carpeta donde se encuentran los datasets
    LOCAL_DIR = DATOS_DIR
    SCALERS_DIR = '../../SCALERS/'
    ENCODERS_DIR = '../../ENCODERS/'
    OUTPUTS_DIR = '../../OUTPUTS/'

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

Mounted at /content/drive/


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#from tslearn.preprocessing import TimeSeriesScalerMeanVariance
#from tslearn.clustering import TimeSeriesKMeans
from joblib import Parallel, delayed
#from tslearn.clustering import TimeSeriesKMeans
from sklearn.preprocessing import MinMaxScaler
import pickle

In [4]:
path_products = DATOS_DIR+'tb_productos.txt'
path_stocks = DATOS_DIR+'tb_stocks.txt'
path_sells = DATOS_DIR+'sell-in.txt'
path_products_to_predict = DATOS_DIR+'product_id_apredecir201912.txt'

In [5]:
tn_scaler_path=SCALERS_DIR+'scalers.pkl'
encoders_path=ENCODERS_DIR+'encoders.pkl'

In [6]:
categorical_cols = ['cat1', 'cat2', 'cat3', 'brand', 'sku_size']

df_products = pd.read_csv(path_products, sep='\t', dtype={ 'product_id': str, 'sku_size': str })
df_stocks = pd.read_csv(path_stocks, sep='\t', dtype={ 'product_id': str, 'periodo': str })
df_sellin = pd.read_csv(path_sells, sep='\t', dtype={ 'product_id': str, 'customer_id': str, 'periodo': str })
df_products_to_predict = pd.read_csv(path_products_to_predict, dtype={ 'product_id': str })



In [7]:
# 🧹 3. Preprocesamiento
# Convertir periodo a datetime
df_sellin['timestamp'] = pd.to_datetime(df_sellin['periodo'], format='%Y%m')

In [8]:
# Filtrar df_sellin para incluir solo los product_ids_to_predict
df_sellin_filtered = df_sellin[df_sellin['product_id'].astype(int).isin(df_products_to_predict['product_id'])].copy()


In [9]:
df_products

,cat1,cat2,cat3,brand,sku_size,product_id
0,HC,ROPA LAVADO,Liquido,LIMPIEX,900,20280
1,HC,ROPA LAVADO,Liquido,LIMPIEX,450,20180
2,HC,ROPA LAVADO,Liquido,LIMPIEX,120,20332
3,HC,ROPA LAVADO,Liquido,LIMPIEX,450,20222
4,HC,ROPA LAVADO,Liquido,LIMPIEX,900,20288
...,...,...,...,...,...,...
1257,FOODS,SOPAS Y CALDOS,Caldo Cubo,MAGGI2,12,20240
1258,FOODS,SOPAS Y CALDOS,Caldo Cubo,MAGGI2,6,20285
1259,FOODS,SOPAS Y CALDOS,Caldo Cubo,MAGGI2,2,20259
1260,FOODS,SOPAS Y CALDOS,Caldo Cubo,MAGGI2,12,20142


In [10]:
df_products_to_predict

,product_id
0,20001
1,20002
2,20003
3,20004
4,20005
...,...
775,21263
776,21265
777,21266
778,21267


In [69]:
# Filtrar por período 201912
df_201912 = df_sellin[df_sellin['periodo'] == "201912"]

In [70]:
df_agrupado = df_201912.groupby('product_id')['tn'].sum().reset_index()

In [71]:
df_201912['product_id'] = df_201912['product_id'].astype(str)
df_products_to_predict['product_id'] = df_products_to_predict['product_id'].astype(str)

/tmp/ipython-input-71-2934532088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_201912['product_id'] = df_201912['product_id'].astype(str)


In [72]:
productos_filtrados = df_201912[df_201912['product_id'].isin(df_products_to_predict['product_id'])]

In [73]:
df_sellin.columns

Index(['periodo', 'customer_id', 'product_id', 'plan_precios_cuidados',
       'cust_request_qty', 'cust_request_tn', 'tn', 'timestamp'],
      dtype='object')

In [74]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Asegurar que product_id es str en todos
df_sellin['product_id'] = df_sellin['product_id'].astype(str)
df_products['product_id'] = df_products['product_id'].astype(str)
df_products_to_predict['product_id'] = df_products_to_predict['product_id'].astype(str)

# Paso 1: Agregamos tn por product_id y periodo
df_agg = df_sellin.groupby(['product_id', 'periodo'], as_index=False)['tn'].sum()
df_agg = df_agg.sort_values(['product_id', 'periodo'])
df_agg['periodo'] = df_agg['periodo'].astype(int)


In [75]:
# Normalizar datos
from scripts import *

df_datos_norm = scale_tn(df_agg, tn_scaler_path, is_train=True)

In [76]:

# Paso 2: Calcular lags y clase
for i in range(1, 12):
    df_datos_norm[f'tn_{i}'] = df_datos_norm.groupby('product_id')['tn'].shift(i)

df_datos_norm['clase'] = df_datos_norm.groupby('product_id')['tn'].shift(-2)

# Paso 3: Merge con catálogo
df_datos_norm = df_datos_norm.merge(df_products, on='product_id', how='left')

In [77]:
df_datos_norm

,product_id,periodo,tn,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,tn_7,tn_8,tn_9,tn_10,tn_11,clase,cat1,cat2,cat3,brand,sku_size
0,20001,201701,-1.576908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.323111,HC,ROPA LAVADO,Liquido,ARIEL,3000
1,20001,201702,-2.042103,-1.576908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.117043,HC,ROPA LAVADO,Liquido,ARIEL,3000
2,20001,201703,-0.323111,-2.042103,-1.576908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.353285,HC,ROPA LAVADO,Liquido,ARIEL,3000
3,20001,201704,-1.117043,-0.323111,-2.042103,-1.576908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.414052,HC,ROPA LAVADO,Liquido,ARIEL,3000
4,20001,201705,0.353285,-1.117043,-0.323111,-2.042103,-1.576908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.250684,HC,ROPA LAVADO,Liquido,ARIEL,3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31607,21295,201701,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31608,21296,201708,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31609,21297,201701,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31610,21298,201708,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
num_distinct_products = df_datos_norm['product_id'].nunique()
print(f"Number of distinct product_ids in df_agg: {num_distinct_products}")

Number of distinct product_ids in df_agg: 1233


In [79]:
# prompt: como hago para filtrar en df_agg para los product_id que estan en df_products_to_predict

df_agg_filtered = df_datos_norm[df_datos_norm['product_id'].isin(df_products_to_predict['product_id'])]
df_agg_filtered

,product_id,periodo,tn,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,tn_7,tn_8,tn_9,tn_10,tn_11,clase,cat1,cat2,cat3,brand,sku_size
0,20001,201701,-1.576908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.323111,HC,ROPA LAVADO,Liquido,ARIEL,3000
1,20001,201702,-2.042103,-1.576908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.117043,HC,ROPA LAVADO,Liquido,ARIEL,3000
2,20001,201703,-0.323111,-2.042103,-1.576908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.353285,HC,ROPA LAVADO,Liquido,ARIEL,3000
3,20001,201704,-1.117043,-0.323111,-2.042103,-1.576908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.414052,HC,ROPA LAVADO,Liquido,ARIEL,3000
4,20001,201705,0.353285,-1.117043,-0.323111,-2.042103,-1.576908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.250684,HC,ROPA LAVADO,Liquido,ARIEL,3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31575,21276,201908,-0.792584,-1.044397,-0.110851,1.145074,1.360154,1.861848,NaN,NaN,NaN,NaN,NaN,NaN,-0.595870,PC,PIEL1,Cara,NIVEA,140
31576,21276,201909,-0.649760,-0.792584,-1.044397,-0.110851,1.145074,1.360154,1.861848,NaN,NaN,NaN,NaN,NaN,-0.290890,PC,PIEL1,Cara,NIVEA,140
31577,21276,201910,-0.595870,-0.649760,-0.792584,-1.044397,-0.110851,1.145074,1.360154,1.861848,NaN,NaN,NaN,NaN,-0.882724,PC,PIEL1,Cara,NIVEA,140
31578,21276,201911,-0.290890,-0.595870,-0.649760,-0.792584,-1.044397,-0.110851,1.145074,1.360154,1.861848,NaN,NaN,NaN,NaN,PC,PIEL1,Cara,NIVEA,140


In [80]:
num_distinct_products = df_agg_filtered['product_id'].nunique()
print(f"Number of distinct product_ids in df_agg: {num_distinct_products}")

Number of distinct product_ids in df_agg: 780


In [81]:

# Paso 4: Definir mágicos y armar dataset de entrenamiento
magicos = ["20002", "20003", "20006", "20010", "20011", "20018", "20019", "20021",
           "20026", "20028", "20035", "20039", "20042", "20044", "20045", "20046", "20049",
           "20051", "20052", "20053", "20055", "20008", "20001", "20017", "20086", "20180",
           "20193", "20320", "20532", "20612", "20637", "20807", "20838"]


In [82]:

campos = ['tn'] + [f'tn_{i}' for i in range(1, 12)] + ['clase']
train_df = df_agg_filtered[(df_agg_filtered['periodo'] == 201812) & (df_agg_filtered['product_id'].isin(magicos))]
train_df = train_df.dropna(subset=campos)


In [83]:
train_df

,product_id,periodo,tn,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,tn_7,tn_8,tn_9,tn_10,tn_11,clase,cat1,cat2,cat3,brand,sku_size
23,20001,201812,0.300509,1.410563,3.050779,0.137189,1.369561,0.245142,-0.842086,-0.355290,-0.500245,1.559624,-1.206154,-0.779892,-0.473681,HC,ROPA LAVADO,Liquido,ARIEL,3000
59,20002,201812,0.000288,2.528304,1.232111,-0.184029,0.509091,-0.106700,0.081647,0.313846,-0.033910,-0.141889,-0.992597,-0.082002,0.112306,HC,ROPA LAVADO,Liquido,LIMPIEX,3000
95,20003,201812,-0.413873,1.104050,1.473641,0.081046,0.232562,-0.808389,-0.792740,-0.363413,-0.428981,-0.383565,-0.349700,0.064451,-0.453818,FOODS,ADEREZOS,Mayonesa,NATURA,475
203,20006,201812,-1.114409,-0.119746,-0.454704,-0.674474,0.187200,-0.441876,-0.736387,1.749419,1.018049,1.730047,0.017300,-0.722709,-0.662236,HC,VAJILLA,Cristalino,LIMPIEX,750
275,20008,201812,-0.802324,-0.757290,-0.136009,-0.735561,0.004409,-0.174150,0.002834,0.963574,0.862741,1.330212,-0.300520,-0.532581,-0.484265,HC,VAJILLA,Opaco,LIMPIEX,750
370,20010,201812,-1.044380,-0.463437,0.419636,-0.170055,0.286967,-0.834537,-1.317766,-1.302729,-1.550846,0.604132,-0.183998,-0.985251,-0.808644,HC,ROPA LAVADO,Polvo,LIMPIEX,400
371,20010,201812,-1.044380,-0.463437,0.419636,-0.170055,0.286967,-0.834537,-1.317766,-1.302729,-1.550846,0.604132,-0.183998,-0.985251,-0.808644,HC,ROPA LAVADO,Polvo,LIMPIEX,400
419,20011,201812,-0.817268,-1.025014,-1.749089,-1.672103,-1.662520,-0.952713,-0.058905,0.242012,0.753337,0.521207,1.003995,-0.689490,-0.098729,HC,ROPA LAVADO,Polvo,LIMPIEX,3000
635,20017,201812,-0.998767,-0.761009,-0.377308,-0.747906,0.000609,-0.201272,-0.504997,1.371127,1.515986,0.984443,0.025551,-0.718898,-0.571959,HC,VAJILLA,Opaco,LIMPIEX,750
671,20018,201812,-0.319025,0.043411,-1.628390,-0.666057,0.075966,-0.326851,0.432075,0.178045,1.409394,1.148704,-0.227187,-0.190386,-0.798193,HC,VAJILLA,Cristalino,LIMPIEX,750


In [84]:

# Paso 5: Entrenar modelo
X_train = train_df[['tn'] + [f'tn_{i}' for i in range(1, 12)]]
y_train = train_df['clase']

In [85]:
df_check = df_agg_filtered[(df_agg_filtered['periodo'] == "201812") & (df_agg_filtered['product_id'].isin(magicos))]
print("Filas encontradas antes del dropna:", df_check.shape[0])
print(df_check[campos].isna().sum())


Filas encontradas antes del dropna: 0
tn       0
tn_1     0
tn_2     0
tn_3     0
tn_4     0
tn_5     0
tn_6     0
tn_7     0
tn_8     0
tn_9     0
tn_10    0
tn_11    0
clase    0
dtype: int64


In [86]:

modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [87]:
# Paso 6: Predecir para periodo 201901
productos_780 = df_products_to_predict['product_id'].tolist()
df_pred = df_agg_filtered[(df_agg_filtered['periodo'] == 201912) & (df_agg_filtered['product_id'].isin(productos_780))].copy()

campos_tn = ['tn'] + [f'tn_{i}' for i in range(1, 12)]
completos = df_pred.dropna(subset=campos_tn).copy()
incompletos = df_pred[~df_pred.index.isin(completos.index)].copy()

X_completos = completos[campos_tn]
completos['pred_clase'] = modelo.predict(X_completos)
promedio_pred = completos['pred_clase'].mean()
incompletos['pred_clase'] = promedio_pred

resultado_final = pd.concat([completos, incompletos]).sort_index()

In [88]:
coef_df = pd.DataFrame({
    'Variable': ['tn'] + [f'tn_{i}' for i in range(1, 12)],
    'Coeficiente': modelo.coef_
})
print("Intercepto:", modelo.intercept_)
print(coef_df)


Intercepto: -0.23325204529917373
   Variable  Coeficiente
0        tn     0.096711
1      tn_1     0.137613
2      tn_2     0.053502
3      tn_3    -0.116880
4      tn_4    -0.192194
5      tn_5     0.107450
6      tn_6    -0.003234
7      tn_7    -0.042620
8      tn_8     0.121842
9      tn_9    -0.062090
10    tn_10    -0.033233
11    tn_11     0.199008


In [89]:
# Paso 1: Filtrar productos a predecir en periodo 201912
df_pred_201912 = df_agg_filtered[(df_agg_filtered['periodo'] == 201912) & (df_agg_filtered['product_id'].isin(productos_780))].copy()

# Paso 2: Separar completos e incompletos
campos_tn = ['tn'] + [f'tn_{i}' for i in range(1, 12)]
completos_201912 = df_pred_201912.dropna(subset=campos_tn).copy()
incompletos_201912 = df_pred_201912[~df_pred_201912.index.isin(completos_201912.index)].copy()

# Paso 3: Predecir con modelo ya entrenado (coeficientes ya calculados)
X_completos_201912 = completos_201912[campos_tn]
completos_201912['pred_clase'] = modelo.predict(X_completos_201912)

# Paso 4: Calcular promedio de las predicciones
promedio_pred_201912 = completos_201912['pred_clase'].mean()

# Paso 5: Aplicar promedio a incompletos
# Para cada SKU incompleto, buscamos su promedio histórico
def promedio_historico(sku):
    historico = df_agg_filtered[(df_agg_filtered['product_id'] == sku) & (df_agg_filtered['periodo'] < 201912)]
    return historico['tn'].mean()

# Aplicamos la función fila por fila
incompletos_201912['pred_clase'] = incompletos_201912['product_id'].apply(promedio_historico)


# Paso 6: Unir y agregar período destino
resultado_201912 = pd.concat([completos_201912, incompletos_201912]).sort_index()
resultado_201912['periodo_predicho'] = 202002

In [90]:
# Toneladas reales (tn) en 201912 para los 780 productos
real_201912 = df_agg_filtered[(df_agg_filtered['periodo'] == 201912) & (df_agg_filtered['product_id'].isin(productos_780))].copy()

# Separar completos e incompletos
completos_real = real_201912.dropna(subset=campos_tn).copy()
incompletos_real = real_201912[~real_201912.index.isin(completos_real.index)].copy()

# Mostrar resultados
print("Toneladas REALES en 201912 (como en la tabla del profe):")
print(" - Completos:", round(completos_real['tn'].sum(), 2))
print(" - Incompletos:", round(incompletos_real['tn'].sum(), 2))
print(" - Total:", round(real_201912['tn'].sum(), 2))


Toneladas REALES en 201912 (como en la tabla del profe):
 - Completos: -499.45
 - Incompletos: -86.86
 - Total: -586.31


In [91]:
# Mostrar verificación al estilo del profesor
print("Resumen predicción para periodo 202002 (usando datos de 201912):")
print("---------------------------------------------------------------")
print("Registros totales:", resultado_201912.shape[0])
print("Completos:", completos_201912.shape[0])
print("Incompletos:", incompletos_201912.shape[0])
print()
print("Toneladas predichas:")
print(" - Completos:", round(completos_201912['pred_clase'].sum(), 2))
print(" - Incompletos:", round(incompletos_201912['pred_clase'].sum(), 2))
print(" - Total:", round(resultado_201912['pred_clase'].sum(), 2))

Resumen predicción para periodo 202002 (usando datos de 201912):
---------------------------------------------------------------
Registros totales: 790
Completos: 665
Incompletos: 125

Toneladas predichas:
 - Completos: -217.53
 - Incompletos: 14.47
 - Total: -203.06


In [92]:
resultado_201912.nunique()

,0
product_id,780
periodo,1
tn,780
tn_1,780
tn_2,780
tn_3,780
tn_4,771
tn_5,756
tn_6,734
tn_7,718


In [93]:
# prompt: En el dataframe resultado_201912 eliminar los product_id duplicados (dejar solo el primer registro)

resultado_201912 = resultado_201912.drop_duplicates(subset='product_id', keep='first').copy()

In [94]:
resultado_201912

,product_id,periodo,tn,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,tn_7,...,tn_10,tn_11,clase,cat1,cat2,cat3,brand,sku_size,pred_clase,periodo_predicho
35,20001,201912,0.361745,-0.003306,0.555017,0.890079,-0.466022,0.954668,-0.981057,0.787270,...,-0.473681,-0.416942,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000,-0.090479,202002
71,20002,201912,0.260161,1.382609,3.238372,0.269771,-0.652856,0.190536,-0.270387,0.085522,...,0.112306,0.859253,NaN,HC,ROPA LAVADO,Liquido,LIMPIEX,3000,0.532121,202002
107,20003,201912,0.012145,0.205901,0.668036,0.273542,-0.880038,-0.603577,-0.786998,-1.037948,...,-0.453818,0.263117,NaN,FOODS,ADEREZOS,Mayonesa,NATURA,475,-0.064303,202002
143,20004,201912,-0.152344,0.236440,1.776149,0.517625,-0.856179,-0.677329,-0.019978,-0.308636,...,-1.038865,-0.724223,NaN,FOODS,ADEREZOS,Mayonesa,NATURA,240,-0.283795,202002
179,20005,201912,-0.236762,-0.173259,1.638239,1.093426,-0.499639,0.471839,1.078878,1.175830,...,-1.088657,-1.303855,NaN,FOODS,ADEREZOS,Mayonesa,NATURA,120,-0.416187,202002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31459,21263,201912,-0.516487,-0.383069,-0.497457,-0.526070,-0.373554,-0.373621,-0.535518,-0.449812,...,-0.202208,-0.306878,NaN,PC,CABELLO,SHAMPOO,VICHY,250,-0.351544,202002
31498,21265,201912,-0.382399,-0.228067,0.190555,-0.702106,-0.713150,-0.580811,-0.371354,0.841014,...,NaN,NaN,NaN,PC,PIEL1,CUIDADO ESPECIAL,LANCOME,32,0.042489,202002
31508,21266,201912,-0.432199,-0.273838,0.235263,-0.658698,-0.794378,-0.353118,-0.420859,0.812502,...,NaN,NaN,NaN,PC,PIEL1,CUIDADO ESPECIAL,LANCOME,32,0.048022,202002
31518,21267,201912,-1.017256,-0.689840,0.051756,-0.982839,-0.689577,-0.241507,-0.448531,2.000029,...,NaN,NaN,NaN,PC,PIEL1,Cara,NIVEA,250,0.113028,202002


In [95]:
# Convertitr en en dataframe salida el valor de tn a la escala original
tn_scaler_path = SCALERS_DIR+'scalers.pkl'

scalers = {}
with open(tn_scaler_path, 'rb') as f:
    scalers = pickle.load(f)

scaled_tn = []
for product_id, group in resultado_201912.groupby('product_id'):
    group = group.copy()
    if product_id in scalers:
        scaler = scalers[product_id]
        group['tn_unscaled'] = scaler.inverse_transform(group[['pred_clase']])
    else:
        print(f"Warning: No scaler found for product {product_id}")
        group['tn_unscaled'] = 0
    scaled_tn.append(group)

resultado_201912 = pd.concat(scaled_tn, axis=0)

In [96]:
# Formato final: solo product_id y tn
export_kaggle = resultado_201912[['product_id', 'tn_unscaled']].copy()
export_kaggle = export_kaggle.rename(columns={'tn_unscaled': 'tn'})

# Exportar a CSV sin índice
export_kaggle.to_csv('prediccion_regresion2años_exp4.csv', index=False)

print("Archivo 'prediccion_regresion1años.csv' generado en formato correcto para Kaggle.")

Archivo 'prediccion_regresion1años.csv' generado en formato correcto para Kaggle.
